# TavilyMap

[Tavily](https://tavily.com/) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed. Tavily offers four key endpoints, one of which being Map, which traverses websites like a graph and can explore hundreds of paths in parallel with intelligent discovery to generate comprehensive site maps.

This guide provides a quick overview for getting started with the Tavily [tool](/docs/integrations/tools/). For a complete breakdown of the Tavily tool, you can find more detailed documentation in the [API reference](https://v03.api.js.langchain.com/modules/_langchain_tavily.html).

## Overview

### Integration details

| Class | Package | [PY support](https://python.langchain.com/docs/integrations/tools/tavily_search/) | Package latest |
| :--- | :--- | :---: | :---: |
| [TavilyMap](https://api.js.langchain.com/classes/_langchain_tavily.TavilyMap.html) | [`@langchain/tavily`](https://www.npmjs.com/package/@langchain/tavily) | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/tavily?style=flat-square&label=%20&) |

## Setup

The integration lives in the `@langchain/tavily` package, which you can install as shown below:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/tavily @langchain/core
</Npm2Yarn>
```

### Credentials

Set up an API key [here](https://app.tavily.com) and set it as an environment variable named `TAVILY_API_KEY`.

```typescript
process.env.TAVILY_API_KEY = "YOUR_API_KEY"
```

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

## Instantiation

You can import and instantiate an instance of the `TavilyMap` tool like this:

In [1]:
import { TavilyMap } from "@langchain/tavily";

const tool = new TavilyMap({
  maxDepth: 3,
  maxBreadth: 50,
  // limit: 100,
  // allowExternal: false,
});

## Invocation

### [Invoke directly with args](/docs/concepts/tools)

The Tavily map tool accepts the following arguments during invocation:

* `url` (required): A natural language search query

* The following arguments can also be set during invocation : `instructions`, `selectPaths` , `selectDomains`, `excludePaths`, `excludeDomains`, `allowExternal`, `categories`.

In [ ]:
await tool.invoke({
  url: "https://docs.tavily.com"
});

### [Invoke with ToolCall](/docs/concepts/tools)

We can also invoke the tool with a model-generated `ToolCall`, in which case a `ToolMessage` will be returned:

In [ ]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    url: "https://docs.tavily.com"
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}

await tool.invoke(modelGeneratedToolCall)

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [4]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

In [7]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("https://docs.tavily.com");

In [ ]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

## Agents

For guides on how to use LangChain tools in agents, see the [LangGraph.js](https://langchain-ai.github.io/langgraphjs/how-tos/#tool-calling) docs.

## API reference

For detailed documentation of all Tavily Map API features and configurations head to the API reference: 

https://docs.tavily.com/documentation/api-reference/endpoint/map

## Related

* Tool [conceptual guide](https://js.langchain.com/docs/concepts/tools/)

* Tool [how-to guides](https://js.langchain.com/docs/how_to/#tools)